In [ ]:
import joblib
import numpy as np
import pandas as pd
import os
import sys
from pathlib import Path

path = Path(os.getcwd()).resolve().parent.parent
if path not in sys.path:
    sys.path.append(str(path))
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.pipeline import Pipeline
from src.utils.data_loader import load_preprocessed_data
from src.data.save_results import save_model_scores, save_model_predictions

target_variables = [
    "temperature_2m_max",
    "apparent_temperature_min",
    "relative_humidity_2m_max",
    "wind_speed_10m_max",
    "winddirection_10m_dominant",
    "rain_sum",
    "shortwave_radiation_sum"
]
for target_variable in target_variables:
    data = load_preprocessed_data(target_variable)

    DATA = data["DATA"]
    TRAINED_DATA = data["TRAINED_DATA"]
    METHOD = data["METHOD"]

    X_train_selected = data["X_train_selected"]
    X_test_selected = data["X_test_selected"]
    y_train = data["y_train"]
    y_test = data["y_test"]
    preprocessor = data["preprocessor"]

    # Pipeline cho GBR
    gbr_pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("model", GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42))
    ])

    # Đánh giá bằng cross-validation
    tscv = TimeSeriesSplit(n_splits=5)
    cv_scores_gbr = cross_val_score(
        gbr_pipeline, X_train_selected, y_train,
        cv=tscv, scoring='neg_root_mean_squared_error'
    )

    # Huấn luyện và đánh giá trên test set
    gbr_pipeline.fit(X_train_selected, y_train)
    y_pred_gbr = gbr_pipeline.predict(X_test_selected)

    mae_gbr = mean_absolute_error(y_test, y_pred_gbr)
    rmse_gbr = np.sqrt(mean_squared_error(y_test, y_pred_gbr))
    r2_gbr = r2_score(y_test, y_pred_gbr)

    print("[RESULT] Gradient Boosting Regressor - {target_variable}")
    print(f"[CV RMSE]: {-np.mean(cv_scores_gbr):.2f} ± {np.std(cv_scores_gbr):.2f}")
    print(f"[MAE]: {mae_gbr:.2f}")
    print(f"[RMSE]: {rmse_gbr:.2f}")
    print(f"[R2 Score]: {r2_gbr:.2f}")
    
    PATH = DATA / "scores&predictions" / METHOD / target_variable
    save_model_scores("GradientBoostingRegressor(gbr)", mae_gbr, rmse_gbr, r2_gbr, -np.mean(cv_scores_gbr), np.std(cv_scores_gbr), PATH)
    save_model_predictions("GradientBoostingRegressor(gbr)", y_test, y_pred_gbr, PATH)